In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
from pathlib import Path
import pandas as pd
import os
from torch.utils.data import Dataset,DataLoader
from PIL import Image
from torchvision import transforms as T
import torch.nn as nn
import torch
import torch.nn.functional as F
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import numpy as np
from fastprogress.fastprogress import master_bar, progress_bar
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from torchvision import models
import pdb
import albumentations as A
from albumentations.pytorch.transforms import ToTensor
import matplotlib.pyplot as plt
import pickle
import cv2
import time

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=86058e867faee4ee6af2006a11be8372bd0d96b85f0e544aa735a6909d749846
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [2]:
import torch
print(torch.__version__)

1.9.0+cu102


In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
models_path = '/content/drive/MyDrive/aROI_Merged_Data_30_05_2021/weights/'
model_pth_name= "efficientnet-b3_0.9716981132075472_10_3_400-400.pth"
model_name='efficientnet-b3'

test_dir='/content/drive/MyDrive/aROI_Merged_Data_30_05_2021/test/'
sizes=400


In [ ]:
def get_model(model_name,lr=1e-5,wd=0.01,freeze_backbone=False,opt_fn=torch.optim.AdamW,device=None):
    if device is None:
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    if model_name.startswith('dense'):
        model = TeknofestDenseNet(model_name=model_name)
    elif model_name.startswith('eff'):
        model = TeknofestEfficientNet(model_name=model_name)
    elif model_name.startswith('res'):
        model = TeknofestResNet(model_name=model_name)
    if freeze_backbone:
        for parameter in model.backbone.parameters():
            parameter.requires_grad = False
    opt = opt_fn(model.parameters(),lr=lr,weight_decay=wd)
    model = model.to(device)
    return model, opt


class TeknofestEfficientNet(nn.Module):
    def __init__(self,model_name,pool_type=F.adaptive_avg_pool2d):
        super().__init__()
        self.pool_type = pool_type
        self.backbone = EfficientNet.from_pretrained(model_name)
        in_features = getattr(self.backbone,'_fc').in_features
        self.classifier = nn.Linear(in_features,1)

    def forward(self,x):
        features = self.pool_type(self.backbone.extract_features(x),1)
        features = features.view(x.size(0),-1)
        return self.classifier(features)


def get_augs_test(p,height,width):
#     p=0.5
    test_tfms = A.Compose([
        A.Resize(height, width),
        ])
    return test_tfms


class TeknofestDataset(Dataset):
    def __init__(self,im_path,transforms=None,is_test=False):
        # self.df = df
        self.im_path = im_path
        self.transforms = transforms
        self.is_test = is_test
        
    def __getitem__(self,idx):
        img_path = f"{self.im_path}"
        img = Image.open(img_path).convert("RGB")
        if self.transforms:
                img = self.transforms(image=np.array(img))["image"]
        if self.is_test:
            return img
        # target = self.df.iloc[idx]['label']
        # return img, torch.tensor([target],dtype=torch.float32)
    
    def __len__(self):
        # return self.df.shape[0]
        return 1


def get_preds(model,test_ds, test_dl, device=None,tta=1):
    if device is None:
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    preds = np.zeros(len(test_ds))
    for tta_id in range(tta):
        test_preds = []
        model.eval()
        with torch.no_grad():
            for xb in test_dl:
                xb = xb.to(device)
                out = model(xb.permute(0,3,1,2).float())
                out = torch.sigmoid(out)
                test_preds.extend(out.cpu().numpy())
            preds += np.array(test_preds).reshape(-1)
        # print(f'TTA {tta_id}')
    preds /= tta
    return preds

In [ ]:
def predict_image_class(file_path: str, models, weights, sizes) -> str:

    #TODO:
    #all preprocessing & inference steps

    preds=[]
    for i, model in enumerate(models):
        test_ds = TeknofestDataset(im_path=file_path, transforms=get_augs_test(p=0.5, height=sizes[i],width=sizes[i]),is_test=True)
        test_dl = DataLoader(dataset=test_ds, batch_size=1, shuffle=False, num_workers=1)
        preds.append(get_preds(model, test_ds, test_dl, tta=1))
    pred = np.sum([preds[i]*weights[i] for i in range(len(models))])
    if pred<0.5: 
        return 'normal'
    else:
        return 'anormal'

In [ ]:
import glob
import os
from pathlib import Path
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
def test(folder_path,model_pth_name,sizes,model_name):
    effnet3model, effnet3opt = get_model(model_name=model_name,lr=1e-4,wd=1e-4)
    effnet3model.load_state_dict(torch.load(f'{models_path}/{model_pth_name}',map_location=device))
    labels = []
    predictions = []
    for image_path in glob.glob("{}/*".format(test_dir)):
        #print(image_path,image_path.split('/')[-1][0])

        label = "normal" if int(image_path.split('/')[-1][0])==0 else "anormal"
        
        # Predict Image Class 
        prediction = predict_image_class(str(image_path),
                                             models=[effnet3model],
                                             weights=[1.0],
                                             sizes=[sizes])
#         print(image_path.split('/')[-1], prediction)
        # Save Predicted and Real label.
        predictions.append(prediction)
        labels.append(label)
        #print(f"label: {label}, pred: {prediction}")
    return labels, predictions
    

print(test_dir)
y_true, y_pred = test(test_dir,
                      model_pth_name=model_pth_name,
                      sizes=sizes,
                      model_name=model_name)

/content/drive/MyDrive/aROI_Merged_Data_30_05_2021/test/


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth



Loaded pretrained weights for efficientnet-b3


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

     anormal       0.68      0.85      0.76       690
      normal       0.89      0.75      0.82      1117

    accuracy                           0.79      1807
   macro avg       0.79      0.80      0.79      1807
weighted avg       0.81      0.79      0.79      1807

